In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('/kaggle/input/training/train.csv')
test=pd.read_csv('/kaggle/input/testing/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.heatmap(train.isnull())

In [ ]:
tt= train[["Air temperature [K]" ,"Process temperature [K]","Rotational speed [rpm]",
        "Torque [Nm]","Tool wear [min]","Machine failure"]]
tt_corr = tt.corr()
sns.heatmap(tt_corr,vmin=-1 , vmax=1, annot=True)

In [ ]:
sns.countplot(x="Type", data=train)

In [ ]:
tt= train[["Air temperature [K]" ,"Process temperature [K]","Rotational speed [rpm]",
        "Torque [Nm]","Tool wear [min]","Machine failure"]]
sns.pairplot(tt ,hue="Machine failure")

In [ ]:
types=pd.get_dummies(train['Type'], drop_first=False)
types.head()

In [ ]:
mapping_boolean = {True: 1, False: 0}
types['H'] = types['H'].map(mapping_boolean)
types['L'] = types['L'].map(mapping_boolean)
types['M'] = types['M'].map(mapping_boolean)
types.head()

In [ ]:
train=pd.concat([train,types], axis=1)
train.head()

In [ ]:
train.drop(['id', 'Product ID', 'Type'], axis=1, inplace=True)
train.head()

In [ ]:
types=pd.get_dummies(test['Type'], drop_first=False)
# types.head()
types['H'] = types['H'].map({True: 1, False: 0})
types['L'] = types['L'].map({True: 1, False: 0})
types['M'] = types['M'].map({True: 1, False: 0})
types.head()


In [ ]:
test=pd.concat([test,types], axis=1)
# test.head()
test.drop(['id', 'Product ID', 'Type'], axis=1, inplace=True)
test.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.metrics import recall_score , f1_score , precision_score ,roc_auc_score   ,accuracy_score
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier

In [ ]:
std =StandardScaler()
test_data_scale = std.fit_transform(test)
test_data_scale

In [ ]:
y=train["Machine failure"]
std =StandardScaler()
train_data_scale = std.fit_transform(train.drop("Machine failure" , axis =1))
train_data_scale

In [ ]:
y=train['Machine failure']
train.drop(['Machine failure'], axis=1, inplace=True)

In [ ]:
model2 = XGBClassifier(n_estimators=10 ,max_depth=3)
model2.fit(train_data_scale,y)
y_predict_train = model2.predict(train_data_scale)
accuracy_score(y,y_predict_train)

In [ ]:
y_predict_test = model2.predict(test_data_scale)
# Print the first 10 indices and predicted values where the output is 1
print("First 10 indices and predicted values where the output is 1:")
count = 0
for i, prediction in enumerate(y_predict_test):
    if prediction == 1:
        print(f"Index {i}: {prediction}")
        count += 1
    if count >= 10:
        break
        
# Print the first 10 indices and predicted values where the output is 1
print("First 10 indices and predicted values where the output is 0:")
count = 0
for i, prediction in enumerate(y_predict_test):
    if prediction == 0:
        print(f"Index {i}: {prediction}")
        count += 1
    if count >= 10:
        break


In [ ]:
total_ones = sum(y_predict_test == 1)
total_records = len(y_predict_test)
percent_ones = (total_ones / total_records) * 100

print(f"Total number of records: {total_records}")
print(f"Total number of Machine Failures in the predicted output: {total_ones}")
print(f"Percentage of Machine Failures in the predicted output: {percent_ones:.2f}%")